# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load messages dataset
messages =pd.read_csv('messages.csv') 
messages.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  
0  Un front froid se retrouve sur Cuba ce matin. ...  direct  
1                 Cyclone nan fini osinon li pa fini  direct  
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct  
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct  
4  facade ouest d Haiti et le reste du pays aujou...  direct

In [3]:
# To check the messages data information
messages.shape

(26248, 4)

In [4]:
# load categories dataset
categories_orig = pd.read_csv('categories.csv')
categories_orig.head()

id                                         categories
0   2  related-1;request-0;offer-0;aid_related-0;medi...
1   7  related-1;request-0;offer-0;aid_related-1;medi...
2   8  related-1;request-0;offer-0;aid_related-0;medi...
3   9  related-1;request-1;offer-0;aid_related-1;medi...
4  12  related-1;request-0;offer-0;aid_related-0;medi...

In [5]:
# To check the categories data information
categories_orig.shape

(26248, 2)

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [6]:
# merge datasets
df = pd.merge(messages,categories_orig, on='id')
df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct   
1                 Cyclone nan fini osinon li pa fini  direct   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct   
4  facade ouest d Haiti et le reste du pays aujou...  direct   

                                          categories  
0  related-1;request-0;offer-0;aid_related-0;medi...  
1  related-1;request-0;offer-0;aid_related-1;medi...  
2  related-1;request-0;offer-0;aid_related-0;medi...  
3  related-1;request-1;offer-0;aid_related-1;medi...  
4  related-1;request-0;offer-0;aid_related-0;medi...

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [7]:
# create a dataframe of the 36 individual category columns
categories = categories_orig.categories.str.split(';', expand=True)
categories.head()

0          1        2              3               4   \
0  related-1  request-0  offer-0  aid_related-0  medical_help-0   
1  related-1  request-0  offer-0  aid_related-1  medical_help-0   
2  related-1  request-0  offer-0  aid_related-0  medical_help-0   
3  related-1  request-1  offer-0  aid_related-1  medical_help-0   
4  related-1  request-0  offer-0  aid_related-0  medical_help-0   

                   5                    6           7           8   \
0  medical_products-0  search_and_rescue-0  security-0  military-0   
1  medical_products-0  search_and_rescue-0  security-0  military-0   
2  medical_products-0  search_and_rescue-0  security-0  military-0   
3  medical_products-1  search_and_rescue-0  security-0  military-0   
4  medical_products-0  search_and_rescue-0  security-0  military-0   

              9        ...                    26                      27  \
0  child_alone-0       ...         aid_centers-0  other_infrastructure-0   
1  child_alone-0       ...         aid_centers-0  other_infrastructure-0   
2  child_alone-0       ...         aid_centers-0  other_infrastructure-0   
3  child_alone-0       ...         aid_centers-0  other_infrastructure-0   
4  child_alone-0       ...         aid_centers-0  other_infrastructure-0   

                  28        29       30      31            32      33  \
0  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
1  weather_related-1  floods-0  storm-1  fire-0  earthquake-0  cold-0   
2  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
3  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
4  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   

                34               35  
0  other_weather-0  direct_report-0  
1  other_weather-0  direct_report-0  
2  other_weather-0  direct_report-0  
3  other_weather-0  direct_report-0  
4  other_weather-0  direct_report-0  

[5 rows x 36 columns]

In [8]:
# select the first row of the categories dataframe
row = categories[1:]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames =  row.applymap(lambda s: s[:-2]).iloc[0, :].tolist()
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [9]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

related    request    offer    aid_related    medical_help  \
0  related-1  request-0  offer-0  aid_related-0  medical_help-0   
1  related-1  request-0  offer-0  aid_related-1  medical_help-0   
2  related-1  request-0  offer-0  aid_related-0  medical_help-0   
3  related-1  request-1  offer-0  aid_related-1  medical_help-0   
4  related-1  request-0  offer-0  aid_related-0  medical_help-0   

     medical_products    search_and_rescue    security    military  \
0  medical_products-0  search_and_rescue-0  security-0  military-0   
1  medical_products-0  search_and_rescue-0  security-0  military-0   
2  medical_products-0  search_and_rescue-0  security-0  military-0   
3  medical_products-1  search_and_rescue-0  security-0  military-0   
4  medical_products-0  search_and_rescue-0  security-0  military-0   

     child_alone       ...           aid_centers    other_infrastructure  \
0  child_alone-0       ...         aid_centers-0  other_infrastructure-0   
1  child_alone-0       ...         aid_centers-0  other_infrastructure-0   
2  child_alone-0       ...         aid_centers-0  other_infrastructure-0   
3  child_alone-0       ...         aid_centers-0  other_infrastructure-0   
4  child_alone-0       ...         aid_centers-0  other_infrastructure-0   

     weather_related    floods    storm    fire    earthquake    cold  \
0  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
1  weather_related-1  floods-0  storm-1  fire-0  earthquake-0  cold-0   
2  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
3  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
4  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   

     other_weather    direct_report  
0  other_weather-0  direct_report-0  
1  other_weather-0  direct_report-0  
2  other_weather-0  direct_report-0  
3  other_weather-0  direct_report-0  
4  other_weather-0  direct_report-0  

[5 rows x 36 columns]

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [10]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    
    # convert column from string to numeric
    categories[column] = categories[column].astype(int)
categories.head()

related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone      ...        \
0                  0         0         0            0      ...         
1                  0         0         0            0      ...         
2                  0         0         0            0      ...         
3                  0         0         0            0      ...         
4                  0         0         0            0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0      0     0   
1            0                     0                1       0      1     0   
2            0                     0                0       0      0     0   
3            0                     0                0       0      0     0   
4            0                     0                0       0      0     0   

   earthquake  cold  other_weather  direct_report  
0           0     0              0              0  
1           0     0              0              0  
2           0     0              0              0  
3           0     0              0              0  
4           0     0              0              0  

[5 rows x 36 columns]

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [11]:
# drop the original categories column from `df`
df.drop('categories', axis=1, inplace=True)

df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  
0  Un front froid se retrouve sur Cuba ce matin. ...  direct  
1                 Cyclone nan fini osinon li pa fini  direct  
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct  
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct  
4  facade ouest d Haiti et le reste du pays aujou...  direct

In [12]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct      1.0   
1                 Cyclone nan fini osinon li pa fini  direct      1.0   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct      1.0   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct      1.0   
4  facade ouest d Haiti et le reste du pays aujou...  direct      1.0   

   request  offer  aid_related  medical_help  medical_products      ...        \
0      0.0    0.0          0.0           0.0               0.0      ...         
1      0.0    0.0          1.0           0.0               0.0      ...         
2      0.0    0.0          0.0           0.0               0.0      ...         
3      1.0    0.0          1.0           0.0               1.0      ...         
4      0.0    0.0          0.0           0.0               0.0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0          0.0                   0.0              0.0     0.0    0.0   0.0   
1          0.0                   0.0              1.0     0.0    1.0   0.0   
2          0.0                   0.0              0.0     0.0    0.0   0.0   
3          0.0                   0.0              0.0     0.0    0.0   0.0   
4          0.0                   0.0              0.0     0.0    0.0   0.0   

   earthquake  cold  other_weather  direct_report  
0         0.0   0.0            0.0            0.0  
1         0.0   0.0            0.0            0.0  
2         0.0   0.0            0.0            0.0  
3         0.0   0.0            0.0            0.0  
4         0.0   0.0            0.0            0.0  

[5 rows x 40 columns]

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [13]:
# check number of duplicates
df[df.duplicated(subset='message')].count()

id                        209
message                   209
original                   93
genre                     209
related                   209
request                   209
offer                     209
aid_related               209
medical_help              209
medical_products          209
search_and_rescue         209
security                  209
military                  209
child_alone               209
water                     209
food                      209
shelter                   209
clothing                  209
money                     209
missing_people            209
refugees                  209
death                     209
other_aid                 209
infrastructure_related    209
transport                 209
buildings                 209
electricity               209
tools                     209
hospitals                 209
shops                     209
aid_centers               209
other_infrastructure      209
weather_related           209
floods    

In [14]:
# drop duplicates
df.drop_duplicates(subset='message', inplace=True)

In [15]:
# check number of duplicates
df[df.duplicated(subset='message')].count()

id                        0
message                   0
original                  0
genre                     0
related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                

In [18]:
df

id                                            message  \
0          2  Weather update - a cold front from Cuba that c...   
1          7            Is the Hurricane over or is it not over   
2          8                    Looking for someone but no name   
3          9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4         12  says: west side of Haiti, rest of the country ...   
5         14             Information about the National Palace-   
6         15                     Storm at sacred heart of jesus   
7         16  Please, we need tents and water. We are in Sil...   
8         17    I would like to receive the messages, thank you   
9         18  I am in Croix-des-Bouquets. We have health iss...   
10        20  There's nothing to eat and water, we starving ...   
11        21  I am in Petionville. I need more information r...   
12        22  I am in Thomassin number 32, in the area named...   
13        24  Let's do it together, need food in Delma 75, i...   
14        25  More information on the 4636 number in order f...   
15        26  A Comitee in Delmas 19, Rue ( street ) Janvier...   
16        27  We need food and water in Klecin 12. We are dy...   
17        28  are you going to call me or do you want me to ...   
18        30     I don't understand how to use this thing 4636.   
19        31  I would like to know if the earthquake is over...   
20        32  I would like to know if one of the radio ginen...   
21        33                     I'm in Laplaine, I am a victim   
22        34  There's a lack of water in Moleya, please info...   
23        35  Those people who live at Sibert need food they...   
24        36  I want to say hello, my message is to let you ...   
25        37                 Can you tell me about this service   
26        38  People I'm at Delma 2, we don't anything what ...   
27        39  We are at Gressier we needs assistance right a...   
28        41  How can we get water and food in Fontamara 43 ...   
29        42  We need help. Carrefour has been forgotten com...   
...      ...                                                ...   
26218  30080  Some 200,000 Chechens displaced by Russian mil...   
26219  30081  A general election held in Mongolia in July 20...   
26220  30082  Until water levels fully recede and infrastruc...   
26221  30083  Grawa, her place of birth, has been known for ...   
26222  30084  The families, who were previously living in ca...   
26223  30085  Some 5 or 6 mt of sorghum seed were taken when...   
26224  30086  He highlighted how the government is using the...   
26225  30087  Police have dropped charges against seven men ...   
26226  30088  Polio vaccination teams have repeatedly come u...   
26227  30089  The most commonly reported ailments at this ti...   
26228  30090  We are the military -- we don't do politics, a...   
26229  30091  (a) The second repatriation convoy of Somali r...   
26230  30093  Due to the storm, squalls with rough seas may ...   
26231  30094  WHO and the Ministry of Health will distribute...   
26232  30095  I held my son and tried to climb up to the roo...   
26233  30096  technical personnel for disease outbreak surve...   
26234  30097  Denied an effective voice and largely ignored ...   
26235  30098  In southern Pakistan's Baluchistan Province, U...   
26236  30100  (b) UNHCR expects to resume the facilitated vo...   
26237  30101  At one point, boxes had to be manhandled almos...   
26238  30102  Police said they expected the toll to rise fur...   
26239  30103  ADRA was granted general consultative status b...   
26240  30104  DEC member agencies are urgently seeking to in...   
26241  30105  Enhancing information exchange, delineating cl...   
26242  30106  NB: IFRC is reportedly stationing a heavy lift...   
26243  30107  "The pattern is always the same: steal, loot, ...   
26244  30108  Sirajganj, Bangladesh. When the rain-swollen J...   
26245  30109  Damage to the Nacala rail corridor by the r

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [ ]:
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('clean_messages', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.